In [32]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from openmm.app import PDBFile
import numpy as np
import plotly.graph_objects as go
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 1. Crear una estructura simplificada de caseína
def create_simplified_casein(num_residues=50):
    # Crear una topología y sistema vacíos
    topology = Topology()
    chain = topology.addChain()
    
    # Añadir residuos
    for i in range(num_residues):
        residue = topology.addResidue('ALA', chain)
        if i == 0:
            topology.addAtom('N', element.hydrogen, residue)
        topology.addAtom('CA', element.carbon, residue)
        topology.addAtom('C', element.carbon, residue)
        if i == num_residues-1:
            topology.addAtom('O', element.oxygen, residue)
    
    # Crear el sistema
    system = System()
    
    # Añadir partículas
    for i in range(num_residues*3):  # 3 átomos por residuo simplificado
        system.addParticle(12.0 * amu)  # Masa aproximada
    
    # Fuerza de enlace
    force = HarmonicBondForce()
    system.addForce(force)
    
    # Añadir enlaces
    for i in range(num_residues):
        # Enlaces dentro del residuo
        idx = i*3
        force.addBond(idx, idx+1, 0.15*nanometer, 1000*kilojoule_per_mole/nanometer**2)  # N-CA
        force.addBond(idx+1, idx+2, 0.15*nanometer, 1000*kilojoule_per_mole/nanometer**2)  # CA-C
        
        # Enlace peptídico entre residuos
        if i < num_residues-1:
            force.addBond(idx+2, (i+1)*3, 0.15*nanometer, 1000*kilojoule_per_mole/nanometer**2)  # C-N
    
    return topology, system

# 2. Simular el proceso de desnaturalización por temperatura
def simulate_denaturation(topology, system, temp_start=300*kelvin, temp_end=500*kelvin, steps=1000):
    # Crear el integrador
    integrator = LangevinMiddleIntegrator(temp_start, 1/picosecond, 0.002*picoseconds)
    
    # Crear la simulación (ahora con topología)
    simulation = Simulation(topology, system, integrator)
    
    # Posiciones iniciales (línea recta)
    positions = []
    for i in range(system.getNumParticles()):
        positions.append(Vec3(i*0.15, 0, 0))
    simulation.context.setPositions(positions)
    
    # Minimizar energía
    simulation.minimizeEnergy()
    
    # Simular aumento gradual de temperatura
    temps = np.linspace(temp_start.value_in_unit(kelvin), temp_end.value_in_unit(kelvin), 10)
    trajectory = []
    
    for temp in temps:
        integrator.setTemperature(temp*kelvin)
        simulation.step(steps//len(temps))
        
        # Obtener posiciones
        state = simulation.context.getState(getPositions=True)
        pos = state.getPositions(asNumpy=True).value_in_unit(nanometer)
        trajectory.append(pos)
    
    return np.array(trajectory)

# 3. Visualización con Plotly
def visualize_denaturation(trajectory):
    fig = go.Figure()
    
    # Añadir frames para la animación
    frames = []
    for i, pos in enumerate(trajectory):
        # Mostrar solo los carbonos alfa para simplificar
        ca_indices = [1 + 3*j for j in range(len(pos)//3)]
        ca_pos = pos[ca_indices]
        
        frames.append(go.Frame(
            data=[go.Scatter3d(
                x=ca_pos[:,0], y=ca_pos[:,1], z=ca_pos[:,2],
                mode='lines+markers',
                marker=dict(size=5, color='blue'),
                line=dict(width=3, color='blue')
            )],
            name=f"Frame {i}"
        ))
    
    # Añadir datos iniciales
    ca_indices = [1 + 3*j for j in range(len(trajectory[0])//3)]
    ca_pos = trajectory[0][ca_indices]
    fig.add_trace(go.Scatter3d(
        x=ca_pos[:,0], y=ca_pos[:,1], z=ca_pos[:,2],
        mode='lines+markers',
        marker=dict(size=5, color='blue'),
        line=dict(width=3, color='blue')
    ))
    
    # Configurar animación
    fig.frames = frames
    fig.update_layout(
        title="Desnaturalización de la Caseína por Temperatura",
        scene=dict(
            xaxis=dict(title='X (nm)'),
            yaxis=dict(title='Y (nm)'),
            zaxis=dict(title='Z (nm)'),
            aspectratio=dict(x=1, y=1, z=1)
        ),
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 100, "redraw": True}}])]
        )]
    )
    
    fig.show()

# Ejecutar la simulación
if __name__ == "__main__":
    print("Creando sistema de caseína simplificado...")
    casein_topology, casein_system = create_simplified_casein(20)  # Reducido a 20 residuos para mayor velocidad
    
    print("Simulando desnaturalización por temperatura...")
    trajectory = simulate_denaturation(casein_topology, casein_system)
    
    print("Generando visualización...")
    visualize_denaturation(trajectory)

Creando sistema de caseína simplificado...
Simulando desnaturalización por temperatura...
Generando visualización...


In [29]:
from openmm.app import *
from openmm import *
from openmm.unit import *
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.distance import pdist
import logging

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class CaseinSimulation:
    """Clase mejorada para simular la desnaturalización de caseína"""
    
    def __init__(self, num_residues=50, include_secondary_structure=True):
        self.num_residues = num_residues
        self.include_secondary_structure = include_secondary_structure
        self.topology = None
        self.system = None
        self.trajectory_data = []
        
    def create_realistic_casein(self):
        """Crear una estructura más realista de caseína con diferentes aminoácidos"""
        logger.info(f"Creando estructura de caseína con {self.num_residues} residuos")
        
        # Secuencia simplificada de aminoácidos comunes en caseína
        casein_sequence = ['SER', 'GLU', 'ASP', 'PRO', 'LYS', 'ALA', 'VAL', 'LEU', 'ILE', 'PHE']
        
        self.topology = Topology()
        chain = self.topology.addChain()
        
        # Crear residuos con diferentes aminoácidos
        residues = []
        atom_count = 0
        
        for i in range(self.num_residues):
            aa_type = casein_sequence[i % len(casein_sequence)]
            residue = self.topology.addResidue(aa_type, chain)
            residues.append(residue)
            
            # Añadir átomos principales de forma consistente
            if i == 0:
                # Primer residuo: N-CA-C
                self.topology.addAtom('N', element.nitrogen, residue)
                self.topology.addAtom('CA', element.carbon, residue)
                self.topology.addAtom('C', element.carbon, residue)
                atom_count += 3
            elif i == self.num_residues - 1:
                # Último residuo: CA-C-O
                self.topology.addAtom('CA', element.carbon, residue)
                self.topology.addAtom('C', element.carbon, residue)
                self.topology.addAtom('O', element.oxygen, residue)
                atom_count += 3
            else:
                # Residuos intermedios: CA-C
                self.topology.addAtom('CA', element.carbon, residue)
                self.topology.addAtom('C', element.carbon, residue)
                atom_count += 2
        
        # Crear sistema con fuerzas más realistas
        self.system = System()
        
        # Añadir partículas con masas específicas
        masses = {'N': 14.0, 'CA': 12.0, 'C': 12.0, 'O': 16.0}
        self.atom_types = []
        
        for i in range(self.num_residues):
            if i == 0:
                # Primer residuo: N-CA-C
                self.system.addParticle(masses['N'] * amu)
                self.atom_types.append('N')
                self.system.addParticle(masses['CA'] * amu)
                self.atom_types.append('CA')
                self.system.addParticle(masses['C'] * amu)
                self.atom_types.append('C')
            elif i == self.num_residues - 1:
                # Último residuo: CA-C-O
                self.system.addParticle(masses['CA'] * amu)
                self.atom_types.append('CA')
                self.system.addParticle(masses['C'] * amu)
                self.atom_types.append('C')
                self.system.addParticle(masses['O'] * amu)
                self.atom_types.append('O')
            else:
                # Residuos intermedios: CA-C
                self.system.addParticle(masses['CA'] * amu)
                self.atom_types.append('CA')
                self.system.addParticle(masses['C'] * amu)
                self.atom_types.append('C')
        self._add_forces()
        
        return self.topology, self.system
    
    def _add_forces(self):
        """Añadir fuerzas más realistas al sistema"""
        
        # 1. Fuerzas de enlace covalente
        bond_force = HarmonicBondForce()
        bond_strengths = {
            'N-CA': (0.147*nanometer, 2500*kilojoule_per_mole/nanometer**2),
            'CA-C': (0.153*nanometer, 2500*kilojoule_per_mole/nanometer**2),
            'C-N': (0.133*nanometer, 3000*kilojoule_per_mole/nanometer**2)
        }
        
        # Crear índices de átomos correctamente
        atom_index = 0
        for i in range(self.num_residues):
            if i == 0:
                # Primer residuo: N-CA-C
                n_idx = atom_index
                ca_idx = atom_index + 1
                c_idx = atom_index + 2
                
                # Enlaces dentro del residuo
                bond_force.addBond(n_idx, ca_idx, *bond_strengths['N-CA'])
                bond_force.addBond(ca_idx, c_idx, *bond_strengths['CA-C'])
                
                atom_index += 3
            elif i == self.num_residues - 1:
                # Último residuo: CA-C-O
                ca_idx = atom_index
                c_idx = atom_index + 1
                o_idx = atom_index + 2
                
                # Enlaces dentro del residuo
                bond_force.addBond(ca_idx, c_idx, *bond_strengths['CA-C'])
                
                # Enlace peptídico con residuo anterior
                prev_c_idx = atom_index - 1
                bond_force.addBond(prev_c_idx, ca_idx, *bond_strengths['C-N'])
                
                atom_index += 3
            else:
                # Residuos intermedios: CA-C
                ca_idx = atom_index
                c_idx = atom_index + 1
                
                # Enlaces dentro del residuo
                bond_force.addBond(ca_idx, c_idx, *bond_strengths['CA-C'])
                
                # Enlace peptídico con residuo anterior
                prev_c_idx = atom_index - 1
                bond_force.addBond(prev_c_idx, ca_idx, *bond_strengths['C-N'])
                
                atom_index += 2
        
        self.system.addForce(bond_force)
        
        # 2. Fuerzas de ángulo (simplificadas para evitar errores de índice)
        angle_force = HarmonicAngleForce()
        
        # Solo añadir ángulos básicos N-CA-C para el primer residuo
        if self.num_residues > 0:
            angle_force.addAngle(0, 1, 2, 1.94, 500*kilojoule_per_mole/radian**2)
        
        self.system.addForce(angle_force)
        
        # 3. Fuerzas de Van der Waals (Lennard-Jones) - simplificadas
        lj_force = NonbondedForce()
        lj_force.setNonbondedMethod(NonbondedForce.CutoffNonPeriodic)
        lj_force.setCutoffDistance(1.0*nanometer)
        
        # Parámetros LJ simplificados - todos los átomos como carbono
        for i in range(self.system.getNumParticles()):
            lj_force.addParticle(0.0, 0.35*nanometer, 0.46*kilojoule_per_mole)
        
        self.system.addForce(lj_force)
        
        # 4. Estructura secundaria simplificada (opcional)
        if self.include_secondary_structure and self.num_residues > 4:
            hbond_force = HarmonicBondForce()
            # Enlaces de hidrógeno muy débiles cada 4 residuos
            for i in range(0, min(self.num_residues-4, 3)):  # Máximo 3 enlaces H
                try:
                    # Índices simplificados
                    idx1 = 2  # C del primer residuo
                    idx2 = 3 + (i+1)*2  # CA de residuo posterior
                    if idx2 < self.system.getNumParticles():
                        hbond_force.addBond(idx1, idx2, 0.3*nanometer, 
                                          20*kilojoule_per_mole/nanometer**2)
                except:
                    pass  # Ignorar errores de índice
            
            self.system.addForce(hbond_force)
    
    def simulate_temperature_denaturation(self, temp_start=300*kelvin, temp_end=400*kelvin, 
                                        temp_steps=20, simulation_steps=2000):
        """Simular desnaturalización con análisis detallado"""
        logger.info(f"Iniciando simulación: {temp_start} → {temp_end}")
        
        # Crear integrador más estable
        integrator = LangevinMiddleIntegrator(temp_start, 5/picosecond, 0.001*picoseconds)
        simulation = Simulation(self.topology, self.system, integrator)
        
        # Posiciones iniciales más realistas (α-hélice parcial)
        positions = self._generate_initial_positions()
        simulation.context.setPositions(positions)
        
        # Minimizar energía
        logger.info("Minimizando energía inicial...")
        simulation.minimizeEnergy(maxIterations=1000)
        
        # Arrays para datos de análisis
        temperatures = np.linspace(temp_start.value_in_unit(kelvin), 
                                 temp_end.value_in_unit(kelvin), temp_steps)
        trajectory = []
        energies = []
        rg_values = []  # Radio de giro
        end_to_end_distances = []
        
        for i, temp in enumerate(temperatures):
            logger.info(f"Simulando a {temp:.1f}K ({i+1}/{len(temperatures)})")
            
            integrator.setTemperature(temp*kelvin)
            simulation.step(simulation_steps // temp_steps)
            
            # Obtener estado actual
            state = simulation.context.getState(getPositions=True, getEnergy=True)
            positions = state.getPositions(asNumpy=True).value_in_unit(nanometer)
            energy = state.getPotentialEnergy().value_in_unit(kilojoule_per_mole)
            
            # Calcular métricas estructurales
            rg = self._calculate_radius_of_gyration(positions)
            end_to_end = self._calculate_end_to_end_distance(positions)
            
            # Almacenar datos
            trajectory.append(positions.copy())
            energies.append(energy)
            rg_values.append(rg)
            end_to_end_distances.append(end_to_end)
        
        # Almacenar datos de análisis
        self.trajectory_data = {
            'positions': np.array(trajectory),
            'temperatures': temperatures,
            'energies': np.array(energies),
            'radius_of_gyration': np.array(rg_values),
            'end_to_end_distance': np.array(end_to_end_distances)
        }
        
        return self.trajectory_data
    
    def _generate_initial_positions(self):
        """Generar posiciones iniciales en forma simplificada"""
        positions = []
        
        # Parámetros básicos
        spacing = 0.15  # nm entre átomos
        
        atom_index = 0
        for i in range(self.num_residues):
            if i == 0:
                # Primer residuo: N-CA-C
                positions.append(Vec3(atom_index * spacing, 0, 0))
                positions.append(Vec3((atom_index + 1) * spacing, 0, 0))
                positions.append(Vec3((atom_index + 2) * spacing, 0, 0))
                atom_index += 3
            elif i == self.num_residues - 1:
                # Último residuo: CA-C-O
                positions.append(Vec3(atom_index * spacing, 0, 0))
                positions.append(Vec3((atom_index + 1) * spacing, 0, 0))
                positions.append(Vec3((atom_index + 2) * spacing, 0, 0))
                atom_index += 3
            else:
                # Residuos intermedios: CA-C
                positions.append(Vec3(atom_index * spacing, 0, 0))
                positions.append(Vec3((atom_index + 1) * spacing, 0, 0))
                atom_index += 2
        
        return positions
    
    def _calculate_radius_of_gyration(self, positions):
        """Calcular radio de giro (Rg) de la proteína"""
        # Usar todas las posiciones para el cálculo
        if len(positions) == 0:
            return 0.0
            
        center_of_mass = np.mean(positions, axis=0)
        rg_squared = np.mean(np.sum((positions - center_of_mass)**2, axis=1))
        return np.sqrt(rg_squared)
    
    def _calculate_end_to_end_distance(self, positions):
        """Calcular distancia extremo a extremo"""
        if len(positions) < 2:
            return 0.0
        start_pos = positions[0]
        end_pos = positions[-1]
        return np.linalg.norm(end_pos - start_pos)
    
    def create_3d_visualization(self):
        """Crear visualización 3D únicamente"""
        if not self.trajectory_data:
            raise ValueError("No hay datos de simulación. Ejecute simulate_temperature_denaturation() primero.")
        
        # Crear figura 3D
        fig = go.Figure()
        
        positions = self.trajectory_data['positions']
        
        # Estructura inicial
        initial_pos = positions[0]
        fig.add_trace(
            go.Scatter3d(
                x=initial_pos[:, 0], 
                y=initial_pos[:, 1], 
                z=initial_pos[:, 2],
                mode='lines+markers',
                marker=dict(size=6, color='blue', opacity=0.8),
                line=dict(width=4, color='blue'),
                name='Estructura Inicial (300K)'
            )
        )
        
        # Estructura final (desnaturalizada)
        final_pos = positions[-1]
        fig.add_trace(
            go.Scatter3d(
                x=final_pos[:, 0], 
                y=final_pos[:, 1], 
                z=final_pos[:, 2],
                mode='lines+markers',
                marker=dict(size=6, color='red', opacity=0.8),
                line=dict(width=4, color='red'),
                name='Estructura Desnaturalizada (400K)'
            )
        )
        
        # Configurar layout
        fig.update_layout(
            title={
                'text': "Desnaturalización de Caseína - Visualización 3D",
                'x': 0.5,
                'font': {'size': 18}
            },
            scene=dict(
                xaxis_title="X (nm)",
                yaxis_title="Y (nm)",
                zaxis_title="Z (nm)",
                bgcolor="rgba(0,0,0,0)",
                xaxis=dict(backgroundcolor="rgba(0,0,0,0)", gridcolor="lightgray"),
                yaxis=dict(backgroundcolor="rgba(0,0,0,0)", gridcolor="lightgray"),
                zaxis=dict(backgroundcolor="rgba(0,0,0,0)", gridcolor="lightgray")
            ),
            width=800,
            height=600,
            showlegend=True,
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="left",
                x=0.01
            )
        )
        
        return fig
    
    def generate_analysis_report(self):
        """Generar reporte de análisis de la simulación"""
        if not self.trajectory_data:
            return "No hay datos de simulación disponibles."
        
        data = self.trajectory_data
        
        # Calcular cambios porcentuales
        rg_change = ((data['radius_of_gyration'][-1] - data['radius_of_gyration'][0]) / 
                     data['radius_of_gyration'][0]) * 100
        
        energy_change = data['energies'][-1] - data['energies'][0]
        
        end_to_end_change = ((data['end_to_end_distance'][-1] - data['end_to_end_distance'][0]) / 
                            data['end_to_end_distance'][0]) * 100
        
        report = f"""
REPORTE DE ANÁLISIS DE DESNATURALIZACIÓN DE CASEÍNA
================================================

Parámetros de Simulación:
- Número de residuos: {self.num_residues}
- Rango de temperatura: {data['temperatures'][0]:.1f}K - {data['temperatures'][-1]:.1f}K
- Número de pasos de temperatura: {len(data['temperatures'])}


Interpretación:
"""
        
        if rg_change > 10:
            report += "- La proteína muestra una expansión significativa, indicando desnaturalización.\n"
        elif rg_change > 5:
            report += "- La proteína muestra una expansión moderada.\n"
        else:
            report += "- La proteína mantiene una estructura compacta.\n"
        
        if energy_change > 0:
            report += "- El aumento de energía indica desestabilización de la estructura.\n"
        else:
            report += "- La energía se mantiene estable o disminuye.\n"
        
        return report

def main():
    """Función principal para ejecutar la simulación mejorada"""
    
    # Crear instancia de simulación
    sim = CaseinSimulation(num_residues=30, include_secondary_structure=True)
    
    # Crear sistema
    topology, system = sim.create_realistic_casein()
    
    # Ejecutar simulación
    trajectory_data = sim.simulate_temperature_denaturation(
        temp_start=300*kelvin,
        temp_end=400*kelvin,
        temp_steps=15,
        simulation_steps=3000
    )
    
    # Crear visualización 3D únicamente
    fig = sim.create_3d_visualization()
    fig.show()
    
    # Generar reporte
    report = sim.generate_analysis_report()
    print(report)
    
    return sim

if __name__ == "__main__":
    casein_sim = main()

INFO:__main__:Creando estructura de caseína con 30 residuos
INFO:__main__:Iniciando simulación: 300 K → 400 K
INFO:__main__:Minimizando energía inicial...
INFO:__main__:Simulando a 300.0K (1/15)
INFO:__main__:Simulando a 307.1K (2/15)
INFO:__main__:Simulando a 314.3K (3/15)
INFO:__main__:Simulando a 321.4K (4/15)
INFO:__main__:Simulando a 328.6K (5/15)
INFO:__main__:Simulando a 335.7K (6/15)
INFO:__main__:Simulando a 342.9K (7/15)
INFO:__main__:Simulando a 350.0K (8/15)
INFO:__main__:Simulando a 357.1K (9/15)
INFO:__main__:Simulando a 364.3K (10/15)
INFO:__main__:Simulando a 371.4K (11/15)
INFO:__main__:Simulando a 378.6K (12/15)
INFO:__main__:Simulando a 385.7K (13/15)
INFO:__main__:Simulando a 392.9K (14/15)
INFO:__main__:Simulando a 400.0K (15/15)



REPORTE DE ANÁLISIS DE DESNATURALIZACIÓN DE CASEÍNA

Parámetros de Simulación:
- Número de residuos: 30
- Rango de temperatura: 300.0K - 400.0K
- Número de pasos de temperatura: 15


Interpretación:
- La proteína mantiene una estructura compacta.
- La energía se mantiene estable o disminuye.



In [36]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from openmm.app import PDBFile
import numpy as np
import plotly.graph_objects as go
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math

# 1. Crear una estructura simplificada de caseína con estructura inicial plegada
def create_simplified_casein(num_residues=30):
    # Crear una topología y sistema vacíos
    topology = Topology()
    chain = topology.addChain()
    
    # Añadir residuos
    for i in range(num_residues):
        residue = topology.addResidue('ALA', chain)
        if i == 0:
            topology.addAtom('N', element.hydrogen, residue)
        topology.addAtom('CA', element.carbon, residue)
        topology.addAtom('C', element.carbon, residue)
        if i == num_residues-1:
            topology.addAtom('O', element.oxygen, residue)
    
    # Crear el sistema
    system = System()
    
    # Añadir partículas
    for i in range(num_residues*3):  # 3 átomos por residuo simplificado
        system.addParticle(12.0 * amu)  # Masa aproximada
    
    # Fuerza de enlace (más flexible para permitir desplegamiento)
    bond_force = HarmonicBondForce()
    system.addForce(bond_force)
    
    # Añadir enlaces
    for i in range(num_residues):
        # Enlaces dentro del residuo
        idx = i*3
        bond_force.addBond(idx, idx+1, 0.15*nanometer, 800*kilojoule_per_mole/nanometer**2)  # N-CA
        bond_force.addBond(idx+1, idx+2, 0.15*nanometer, 800*kilojoule_per_mole/nanometer**2)  # CA-C
        
        # Enlace peptídico entre residuos
        if i < num_residues-1:
            bond_force.addBond(idx+2, (i+1)*3, 0.15*nanometer, 800*kilojoule_per_mole/nanometer**2)  # C-N
    
    # Añadir fuerzas no enlazantes que mantienen la estructura compacta a baja temperatura
    nb_force = NonbondedForce()
    system.addForce(nb_force)
    
    # Añadir parámetros para interacciones débiles entre residuos distantes
    for i in range(num_residues*3):
        nb_force.addParticle(0.0, 0.3*nanometer, 0.1*kilojoule_per_mole)
    
    return topology, system

# 2. Crear posiciones iniciales que imiten la estructura plegada de la imagen
def create_folded_structure(num_residues):
    positions = []
    
    # Crear una estructura plegada con loops y vueltas como en la imagen
    # Usar escalas más grandes para evitar solapamientos
    for i in range(num_residues):
        t = i / max(1, num_residues - 1) * 3 * math.pi  # Parámetro para crear la estructura
        
        # Crear una estructura con múltiples loops y vueltas, con más separación
        if i < num_residues // 3:
            # Primer loop (parte superior izquierda)
            x = 1.0 * math.cos(t * 1.5) + 0.4 * math.sin(t * 2)
            y = 1.0 * math.sin(t * 1.5) + 0.5
            z = 0.3 * math.cos(t * 2.5)
        elif i < 2 * num_residues // 3:
            # Loop central (cruzamiento)
            x = 0.8 * math.cos(t * 0.8) - 0.3
            y = 0.6 * math.sin(t * 1.2)
            z = 0.4 * math.sin(t * 1.8)
        else:
            # Loop final (parte inferior)
            x = 1.2 * math.cos(t * 0.6) + 0.2
            y = 0.8 * math.sin(t * 0.6) - 0.6
            z = 0.3 * math.cos(t * 1.2)
        
        # Añadir los 3 átomos por residuo con separación adecuada
        for j in range(3):
            # Separación mínima entre átomos del mismo residuo
            offset_x = j * 0.15 * math.cos(i * 0.1)
            offset_y = j * 0.15 * math.sin(i * 0.1)
            offset_z = j * 0.1
            positions.append(Vec3(x + offset_x, y + offset_y, z + offset_z))
    
    return positions

# 3. Simular el proceso de desnaturalización gradual
def simulate_denaturation(topology, system, temp_start=300*kelvin, temp_end=500*kelvin, steps=2000):
    # Crear el integrador con más fricción para movimientos más suaves
    integrator = LangevinMiddleIntegrator(temp_start, 2/picosecond, 0.002*picoseconds)
    
    # Crear la simulación
    simulation = Simulation(topology, system, integrator)
    
    # Posiciones iniciales plegadas
    positions = create_folded_structure(system.getNumParticles()//3)
    simulation.context.setPositions(positions)
    
    # Verificar que las posiciones son válidas antes de minimizar
    state = simulation.context.getState(getPositions=True)
    pos_check = state.getPositions(asNumpy=True)
    if np.any(np.isnan(pos_check)):
        print("⚠️ Posiciones iniciales contienen NaN, regenerando...")
        # Crear posiciones más simples si hay problemas
        simple_positions = []
        for i in range(system.getNumParticles()):
            simple_positions.append(Vec3(i*0.2, 0, 0))
        simulation.context.setPositions(simple_positions)
    
    # Minimizar energía muy suavemente
    try:
        simulation.minimizeEnergy(tolerance=10*kilojoule_per_mole/nanometer, maxIterations=100)
        print("✅ Minimización de energía completada")
    except Exception as e:
        print(f"⚠️ Error en minimización: {e}")
        print("Continuando sin minimización...")
    
    # Simular aumento gradual de temperatura con más pasos
    temps = np.linspace(temp_start.value_in_unit(kelvin), temp_end.value_in_unit(kelvin), 20)
    trajectory = []
    
    for i, temp in enumerate(temps):
        integrator.setTemperature(temp*kelvin)
        
        # Hacer pasos más pequeños para evitar inestabilidades
        small_steps = max(50, steps//len(temps)//2)
        
        try:
            simulation.step(small_steps)
            
            # Verificar estabilidad después de cada paso
            state = simulation.context.getState(getPositions=True)
            pos = state.getPositions(asNumpy=True).value_in_unit(nanometer)
            
            if np.any(np.isnan(pos)) or np.any(np.abs(pos) > 50):
                print(f"⚠️ Simulación inestable en temperatura {temp:.0f}K")
                break
                
            trajectory.append(pos)
            
        except Exception as e:
            print(f"⚠️ Error en simulación a {temp:.0f}K: {e}")
            break
    
    if len(trajectory) == 0:
        print("⚠️ Creando trayectoria sintética debido a errores de simulación...")
        trajectory = create_synthetic_trajectory(system.getNumParticles()//3)
    
    return np.array(trajectory)

# Función auxiliar para crear una trayectoria sintética si la simulación falla
def create_synthetic_trajectory(num_residues, num_frames=20):
    trajectory = []
    
    for frame in range(num_frames):
        positions = []
        progress = frame / (num_frames - 1)
        
        for i in range(num_residues):
            # Interpolar entre estructura plegada y desplegada
            if progress < 0.5:
                # Estructura más compacta al inicio
                t = i / max(1, num_residues - 1) * 2 * math.pi
                x = (1 - progress) * (0.8 * math.cos(t * 2) + 0.3 * math.sin(t * 3)) + progress * i * 0.3
                y = (1 - progress) * (0.8 * math.sin(t * 2)) + progress * 0.2 * math.sin(i * 0.5)
                z = (1 - progress) * (0.4 * math.cos(t * 3)) + progress * 0.1 * math.cos(i * 0.3)
            else:
                # Estructura más lineal al final
                factor = (progress - 0.5) * 2
                x = i * 0.3 + factor * i * 0.2
                y = 0.2 * math.sin(i * 0.5) * (1 - factor * 0.8)
                z = 0.1 * math.cos(i * 0.3) * (1 - factor * 0.9)
            
            # Añadir 3 átomos por residuo
            for j in range(3):
                offset_x = j * 0.1
                positions.append([x + offset_x, y, z])
        
        trajectory.append(np.array(positions))
    
    return trajectory

# 4. Visualización mejorada que muestra el desplegamiento realista
def visualize_denaturation(trajectory):
    fig = go.Figure()
    
    # Calcular rangos para la caja
    all_positions = trajectory.reshape(-1, 3)
    margin = 1.5
    x_range = [np.min(all_positions[:, 0]) - margin, np.max(all_positions[:, 0]) + margin]
    y_range = [np.min(all_positions[:, 1]) - margin, np.max(all_positions[:, 1]) + margin]
    z_range = [np.min(all_positions[:, 2]) - margin, np.max(all_positions[:, 2]) + margin]
    
    # Añadir frames para la animación
    frames = []
    for i, pos in enumerate(trajectory):
        # Mostrar solo los carbonos alfa
        ca_indices = [1 + 3*j for j in range(len(pos)//3)]
        ca_pos = pos[ca_indices]
        
        # Cambiar solo el color gradualmente, mantener tamaño constante
        temp_ratio = i / (len(trajectory) - 1)
        
        # Color de azul oscuro a rojo brillante
        if temp_ratio < 0.3:
            color = f'rgb(0, 0, {int(255 - 100 * temp_ratio)})'  # Azul oscuro a azul
        elif temp_ratio < 0.7:
            color = f'rgb({int(255 * (temp_ratio - 0.3) / 0.4)}, 0, {int(255 - 255 * (temp_ratio - 0.3) / 0.4)})'  # Azul a rojo
        else:
            color = f'rgb(255, {int(50 * (temp_ratio - 0.7) / 0.3)}, 0)'  # Rojo oscuro a rojo brillante
        
        # Mantener tamaño constante durante toda la simulación
        line_width = 4  # Grosor constante
        marker_size = 8  # Tamaño constante
        
        temp_actual = 300 + (200 * temp_ratio)
        
        frames.append(go.Frame(
            data=[go.Scatter3d(
                x=ca_pos[:,0], y=ca_pos[:,1], z=ca_pos[:,2],
                mode='lines+markers',
                marker=dict(
                    size=marker_size, 
                    color=color,
                    opacity=0.8
                ),
                line=dict(
                    width=line_width, 
                    color=color
                ),
                name=f'Temperatura: {temp_actual:.0f}K'
            )],
            name=f"Frame {i}"
        ))
    
    # Estado inicial
    ca_indices = [1 + 3*j for j in range(len(trajectory[0])//3)]
    ca_pos = trajectory[0][ca_indices]
    fig.add_trace(go.Scatter3d(
        x=ca_pos[:,0], y=ca_pos[:,1], z=ca_pos[:,2],
        mode='lines+markers',
        marker=dict(size=8, color='darkblue', opacity=0.8),  # Mismo tamaño que el resto
        line=dict(width=4, color='darkblue'),  # Mismo grosor que el resto
        name='Proteína Nativa (300K)'
    ))
    
    # Configurar la visualización
    fig.frames = frames
    fig.update_layout(
        title={
            'text': "Desnaturalización de Caseína: De Estructura Plegada a Lineal<br><sub>Basado en el proceso real de desplegamiento proteico</sub>",
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 16}
        },
        scene=dict(
            xaxis=dict(
                title='X (nm)', 
                range=x_range,
                showgrid=True,
                gridcolor='lightgray',
                gridwidth=1
            ),
            yaxis=dict(
                title='Y (nm)', 
                range=y_range,
                showgrid=True,
                gridcolor='lightgray',
                gridwidth=1
            ),
            zaxis=dict(
                title='Z (nm)', 
                range=z_range,
                showgrid=True,
                gridcolor='lightgray',
                gridwidth=1
            ),
            aspectratio=dict(x=1, y=1, z=0.8),
            bgcolor='rgba(240,240,240,0.1)',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.2)
            )
        ),
        updatemenus=[dict(
            type="buttons",
            direction="left",
            pad={"r": 10, "t": 100},
            showactive=True,
            x=0.01,
            xanchor="right",
            y=0,
            yanchor="top",
            buttons=[
                dict(
                    label="▶ Iniciar Desnaturalización",
                    method="animate",
                    args=[None, {
                        "frame": {"duration": 600, "redraw": True},  # Más lento para apreciar el proceso
                        "fromcurrent": True,
                        "transition": {"duration": 400, "easing": "cubic-in-out"}
                    }]
                ),
                dict(
                    label="⏸ Pausar",
                    method="animate",
                    args=[[None], {
                        "frame": {"duration": 0, "redraw": False},
                        "mode": "immediate",
                        "transition": {"duration": 0}
                    }]
                ),
                dict(
                    label="🔄 Reiniciar",
                    method="animate",
                    args=[[None], {
                        "frame": {"duration": 0, "redraw": True},
                        "mode": "immediate",
                        "transition": {"duration": 0}
                    }]
                )
            ]
        )],
        annotations=[
            dict(
                text="🌡️ Azul: Estructura nativa compacta<br>🔥 Rojo: Proteína desnaturalizada<br>📏 Tamaño constante de aminoácidos",
                showarrow=False,
                xref="paper", yref="paper",
                x=0.02, y=0.98,
                xanchor="left", yanchor="top",
                font=dict(size=12, color="black"),
                bgcolor="rgba(255,255,255,0.8)",
                borderwidth=1
            )
        ],
        width=1000,
        height=800
    )
    
    fig.show()

# Ejecutar la simulación
if __name__ == "__main__":
    print("🧬 Creando proteína de caseína con estructura plegada inicial...")
    casein_topology, casein_system = create_simplified_casein(25)
    
    print("🌡️ Simulando desnaturalización térmica gradual...")
    trajectory = simulate_denaturation(casein_topology, casein_system)
    
    print("📊 Generando visualización del desplegamiento...")
    visualize_denaturation(trajectory)

🧬 Creando proteína de caseína con estructura plegada inicial...
🌡️ Simulando desnaturalización térmica gradual...
✅ Minimización de energía completada
📊 Generando visualización del desplegamiento...
